## Recent Logs

One common way to use Redis to store logs is to store the recent 100 logs in a LIST.

In [11]:
import logging

SEVERITY = {
    logging.DEBUG: 'debug',
    logging.INFO: 'info',
    logging.WARNING: 'warning',
    logging.ERROR: 'error',
    logging.CRITICAL: 'critical'
}

In [9]:
SEVERITY.update((name, name) for name in list(SEVERITY.values()))
SEVERITY

{10: 'debug',
 20: 'info',
 30: 'warning',
 40: 'error',
 50: 'critical',
 'debug': 'debug',
 'info': 'info',
 'warning': 'warning',
 'error': 'error',
 'critical': 'critical'}

In [10]:
def log_recent(conn, name, message, severity=logging.INFO, pipe=None):
    # Turn logging level into simple string.
    severity = str(SEVERITY.get(severity, severity)).lower()
    
    # Create a key that messages will be written to.
    destination = f'recent:{name}:{severity}'
        
    # Add the current time so that we know when the message was sent.
    message = time.asctime() + ' ' + message
    
    # Set up a pipeline so we only need one round trip.
    pipe = pipe or conn.pipeline()
    
    # Add the message to the beginning of the list.
    pipe.lpush(destination, message)
    
    # Trim the log list to only include the most recent 100 messages.
    pipe.ltrim(destination, 0, 99)
    pipe.execute()

## Common Logs

Recent logs does not provide information on how often a particular logs appear, which can be useful to detect issues early. 

To keep track of commonly occuring logs, we can use `ZSET` with the score being the frequency of occurances of the logs. To keep track of the recent common logs, we rotate the records hourly and keep the previous hour's worth of common logs.

In [31]:
from datetime import datetime

def log_common(conn, name, message, severity=logging.INFO, timeout=5):
    # Handle the logging level.
    severity = str(SEVERITY.get(severity, severity)).lower()
    
    # Set up the destination key for keeping recent logs.
    destination = f'common:{name}:{severity}'
        
    # Keep a record of the start of the hour for this set of messages.
    start_key = destination + ':start'
    
    pipe = conn.pipeline()
    
    end = time.time() + timeout
    while time.time() < end:
        try:
            # We will watch the start of the hour key for changes that only happens
            # at the beginning of the hour.
            pipe.watch(start_key)
            now = datetime.utcnow().timetuple()
            hour_start = datetime(*now[:4]).isoformat()
            
            existing = pipe.get(start_key)
            pipe.multi()
            # If the current list of common logs is for a previous hour...
            if existing and existing < hour_start:
                # Move the old common log information to the archive.
                pipe.rename(destination, destination + ':last')
                pipe.rename(start_key, destination + ':pstart')
                
                # Update the staart of the current hour for the common logs.
                pipe.set(start_key, hour_start)
            
            # Increment the common counter.
            pipe.zincrby(destination, message)
            
            # Call the log_recent function to record this.
            log_recent(pipe, name, message, severity, pipe)
            return
        except redis.exceptions.WatchError:
            # If we get error from someone else archiving, try again.
            continue